In [4]:
import matplotlib.pyplot as plt
import numpy as np

class FaceRecognizer:
    def __init__(self, retention):
        self.retention = retention
        self.face_data = None
        self.labels = None
        self.mean_face = None
        self.faces_less_mean = None
        self.eigenfaces = None
        self.weights = None

    def train(self):
        self.load_data()
        self.mean_face = np.average(self.face_data, axis=0)
        self.find_eigenfaces()
        self.weights = np.matmul(self.faces_less_mean, self.eigenfaces)

    def load_data(self):
        raw_data = np.genfromtxt('olivetti_faces_augmented.csv', delimiter=',')
        data_without_headers = np.delete(raw_data, 0, axis=0)
        self.labels = data_without_headers[:, -1]
        self.face_data = np.delete(data_without_headers, -1, axis=1)

    def find_eigenfaces(self):
        self.faces_less_mean = self.face_data - self.mean_face
        U, S, _ = np.linalg.svd(self.faces_less_mean.T)
        total_energy = S.sum()
        retention_candidates = np.where(S.cumsum() / total_energy > self.retention)
        dimensions_to_retain = retention_candidates[0][0]
        self.eigenfaces = U[:, 0:dimensions_to_retain]

    def reconstruct(self, weights):
        return np.matmul(weights, self.eigenfaces.T) + self.mean_face

    def find_best_match(self, face):
        face_less_mean = face - self.mean_face
        face_weights = np.matmul(face_less_mean, self.eigenfaces)
        diffs = self.weights - face_weights
        squared_diffs = diffs ** 2
        match_index = np.argmin(squared_diffs.sum(axis=1))
        return self.reconstruct(self.weights[match_index]), self.labels[match_index]


if __name__ == '__main__':
    import utils

    fr = FaceRecognizer(0.9)
    fr.train()

    search_vector = utils.image_to_vector('img/test_input.jpg')
    match, match_label = fr.find_best_match(search_vector)
    plt.imsave('img/match.jpg', match.reshape(64, 64), cmap='gray')

AttributeError: module 'utils' has no attribute 'image_to_vector'

In [3]:
import numpy as np
from PIL import Image

def resize_images(images, desired_dimensions):
    for name in images:
        im = Image.open(name)
        out = im.resize(desired_dimensions)
        out.save(f'formatted_{name}')

def rgb2gray(img_rgb):
    return img_rgb[:, :, 0] * 0.2989 + img_rgb[:, :, 1] * 0.5870 + img_rgb[:, :, 2] * 0.1140

def image_to_vector(filepath):
    img_data = np.asarray(Image.open(filepath))
    if len(img_data.shape) > 2:
        img_data = rgb2gray(img_data) / 255.0
    return img_data.reshape(-1)